# Project 15: Text2SQL via Prompt Engineering

# Text-to-SQL: Bridging the Gap Between Human Language and Databases


Text-to-SQL, also known as Natural Language to SQL (NL2SQL), is a rapidly evolving technology that translates natural, everyday language into Structured Query Language (SQL) commands. This innovative approach empowers users to interact with and retrieve data from databases simply by asking questions in plain English, eliminating the need for specialized knowledge of complex SQL syntax.

At its core, Text-to-SQL acts as an intelligent translator. It leverages the power of artificial intelligence, particularly **Natural Language Processing (NLP)** and sophisticated **AI models**, to understand the user's intent and generate the corresponding SQL query. This process allows individuals without a technical background to explore and analyze data, thereby democratizing data access within an organization.



# Submission Details

Can you create dataset for employees with at least 2 tables, and share the results in a notebook?

# **Retrieving Data**

 Created dataset using Mockaroo

In [ ]:
! curl "https://api.mockaroo.com/api/f0a22b30?count=1000&key=34e1ad80" > "employees.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 96759    0 96759    0     0   7362      0 --:--:--  0:00:13 --:--:--  8160


In [ ]:
! curl "https://api.mockaroo.com/api/4f7a18c0?count=10&key=34e1ad80" > "departments.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   265    0   265    0     0    415      0 --:--:-- --:--:-- --:--:--   416


# Setting Up Database

In [ ]:
import sqlite3
import pandas as pd
import os

In [ ]:
# Define SQL schemas for creating tables
departments_schema = """
CREATE TABLE IF NOT EXISTS departments (
    DepartmentID INTEGER PRIMARY KEY AUTOINCREMENT,
    DepartmentName VARCHAR(100),
    Location VARCHAR(100)
);
"""

employees_schema = """
CREATE TABLE IF NOT EXISTS employees (
    EmployeeID INTEGER PRIMARY KEY AUTOINCREMENT,
    FirstName VARCHAR(50) NOT NULL,
    LastName VARCHAR(50) NOT NULL,
    Email VARCHAR(100),
    PhoneNumber VARCHAR(20),
    HireDate DATE NOT NULL,
    JobTitle VARCHAR(100),
    Salary DECIMAL(10, 2),
    DepartmentID INT,
    FOREIGN KEY (DepartmentID) REFERENCES Departments(DepartmentID)
);
"""



In [ ]:
db_name = 'employee.db'
if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Removed existing database '{db_name}'.")

Removed existing database 'employee.db'.


In [ ]:
COLUMN_DATA_TYPES = {
    'employees': {
        'EmployeeID': 'int64',
        'FirstName': 'object',
        'LastName': 'object',
        'Email': 'object',
        'PhoneNumber': 'object',
        'HireDate': 'object',
        'JobTitle': 'object',
        'Salary': 'float64',
        'DepartmentID': 'int64'
    },
    'departments': {
        'DepartmentID': 'int64',
        'DepartmentName': 'object',
        'Location': 'object'
    }
}

# --- Database setup ---
db_name = 'employee.db'

# Remove existing database file if it exists
if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Removed existing database '{db_name}'.")


conn = None  # Initialize connection to None

try:
    # Establish a connection to the SQLite database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    print(f"Database '{db_name}' created and connected successfully. ✅")

    # Create tables
    cursor.execute(departments_schema)
    cursor.execute(employees_schema)
    print("Tables 'employees' and 'departments' created successfully.")

    # --- Load data from CSV files into the tables using pandas ---
    csv_to_table_map = {
        '/content/employees.csv': 'employees',
        '/content/departments.csv': 'departments' # Corrected file extension
    }

    for csv_file, table_name in csv_to_table_map.items():
        if os.path.exists(csv_file):
            print(f"\nProcessing '{csv_file}' for table '{table_name}'...")

            # Read the CSV file into a pandas DataFrame
            try:
                df = pd.read_csv(csv_file, on_bad_lines='skip') # Added error handling
            except Exception as e:
                print(f"  - Error reading CSV file '{csv_file}': {e}")
                continue # Skip to the next file if reading fails


            # 1. Get the expected schema for the current table
            expected_schema = COLUMN_DATA_TYPES[table_name]
            expected_cols = list(expected_schema.keys())

            # 2. Handle missing/extra columns
            # Drop columns from DataFrame that are not in the schema
            df = df[df.columns.intersection(expected_cols)]

            # Add any missing columns and fill with None (which becomes NULL in SQL)
            for col in expected_cols:
                if col not in df.columns:
                    df[col] = None

            # 3. Reorder columns to match the defined schema exactly
            df = df[expected_cols]

            # 4. Enforce data types
            for col, dtype in expected_schema.items():
                if 'datetime' in dtype:
                    # Use pd.to_datetime for date/time columns, coercing errors to NaT (Not a Time)
                    df[col] = pd.to_datetime(df[col], errors='coerce')
                else:
                    # Use astype for other columns, handling potential conversion errors
                    try:
                        df[col] = df[col].astype(dtype)
                    except (ValueError, TypeError) as e:
                        print(f"  - Warning: Could not convert column '{col}' to {dtype}. Error: {e}. Leaving as is.")

            # --- Handle duplicate EmployeeIDs for employees table ---
            if table_name == 'employees':
                # Create a temporary table
                temp_table_name = f"temp_{table_name}"
                df.to_sql(temp_table_name, conn, if_exists='replace', index=False)
                print(f"  -> Data from '{csv_file}' loaded into temporary table '{temp_table_name}'.")

                # Insert from temporary table into main table, ignoring conflicts on EmployeeID
                insert_query = f"""
                INSERT OR IGNORE INTO {table_name} ({', '.join(expected_cols)})
                SELECT {', '.join(expected_cols)}
                FROM {temp_table_name};
                """
                cursor.execute(insert_query)
                print(f"  -> Data from '{temp_table_name}' inserted into '{table_name}', ignoring duplicate EmployeeIDs.")

                # Drop the temporary table
                drop_temp_query = f"DROP TABLE {temp_table_name};"
                cursor.execute(drop_temp_query)
                print(f"  -> Temporary table '{temp_table_name}' dropped.")
            else:
                # Use the to_sql method to insert the cleaned DataFrame for other tables
                df.to_sql(table_name, conn, if_exists='append', index=False)
                print(f"  -> Data from '{csv_file}' loaded into '{table_name}' table successfully.")


    # Commit the changes to the database
    conn.commit()
    print("\nData committed to the database successfully. 🎉")

except sqlite3.Error as e:
    print(f"Database error: {e}")
except pd.errors.EmptyDataError as e:
    print(f"Pandas error: {e}. One of the CSV files might be empty.")
except KeyError as e:
    print(f"Schema definition error: A column is missing from the TABLE_DATA_TYPES dictionary: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    # Close the connection if it was established
    if conn:
        conn.close()
        print("Database connection closed.")

Database 'employee.db' created and connected successfully. ✅
Tables 'employees' and 'departments' created successfully.

Processing '/content/employees.csv' for table 'employees'...
  -> Data from '/content/employees.csv' loaded into temporary table 'temp_employees'.
  -> Data from 'temp_employees' inserted into 'employees', ignoring duplicate EmployeeIDs.
  -> Temporary table 'temp_employees' dropped.

Processing '/content/departments.csv' for table 'departments'...
  -> Data from '/content/departments.csv' loaded into 'departments' table successfully.

Data committed to the database successfully. 🎉
Database connection closed.


# Setting up your free API Key using Google's AI Studio

https://aistudio.google.com/


And the KEY as Secrets of colab notebook

In [ ]:
!pip install google-genai

In [ ]:
from google import genai
from google.colab import userdata

In [ ]:
genai_client = genai.Client(api_key=userdata.get('21DaysP15'))

In [ ]:
prompt = """

###ROLE###
You are a highly skilled Text-to-SQL translator with expertise in SQL syntax, database schema interpretation, and natural language understanding. You generate syntactically correct and semantically accurate SQL queries based on user input and a given database schema.

###CONTEXT###
The user is working with a relational database for an Employee management system. The database includes three main tables: `employees`, and `departments`. The goal is to allow users to input natural language queries (in English), and have the model return equivalent SQL statements that accurately extract the requested data using the given schema.

Here is the full schema:


**Departments Table**
```sql
CREATE TABLE IF NOT EXISTS departments (
    DepartmentID INTEGER PRIMARY KEY AUTOINCREMENT,
    DepartmentName VARCHAR(100),
    Location VARCHAR(100)
);
````

**Employees Table**

```sql
CREATE TABLE IF NOT EXISTS employees (
    EmployeeID INTEGER PRIMARY KEY AUTOINCREMENT,
    FirstName VARCHAR(50) NOT NULL,
    LastName VARCHAR(50) NOT NULL,
    Email VARCHAR(100),
    PhoneNumber VARCHAR(20),
    HireDate DATE NOT NULL,
    JobTitle VARCHAR(100),
    Salary DECIMAL(10, 2),
    DepartmentID INT,
    FOREIGN KEY (DepartmentID) REFERENCES Departments(DepartmentID)
);
```

###TASK###
Your task is to:

1. Read a natural language query about the **Employee** or **Department** data.
2. Interpret the user's intent based on the schema provided.
3. Generate a valid SQL `SELECT` query that returns the expected result.
4. Ensure correct table joins, column selection, filtering, and grouping as necessary.
5. Handle aggregate functions (e.g., `COUNT`, `AVG`, `SUM`, `MIN`, `MAX`) where appropriate.
6. Disambiguate user terms based on schema details (e.g., "division" → `departments`, "employee name" → `employees.FirstName` and `employees.LastName`, etc.).

---

###CONSTRAINTS###

* Only return a valid SQL query as output — no explanations or extra text.
* The user is using a **SQLite** database — respond with correct and valid SQLite syntax.
* Use aliases (`AS`) for column names only when the original name is ambiguous.
* Do not create, modify, or drop any tables.
* Do not assume the existence of tables or columns not provided in the schema.
* Avoid subqueries unless absolutely necessary for correctness or performance.
* Prefer readability: indent `JOIN`, `WHERE`, `GROUP BY`, `ORDER BY`, and `LIMIT` clauses properly.

---

###EXAMPLES###

**Input:**
"Show me all employee names and their department names."

**Output:**
```sql
SELECT e.FirstName, e.LastName, d.DepartmentName
FROM employees AS e
JOIN departments AS d ON e.DepartmentID = d.DepartmentID;

```

**Input:**
"List all employees who work in the 'Human Resources' department."

**Output:**
```sql
SELECT e.FirstName, e.LastName
FROM employees AS e
JOIN departments AS d ON e.DepartmentID = d.DepartmentID
WHERE d.DepartmentName = 'Human Resources';

```


**Input:**
"Find the total number of employees in each department."

**Output:**
```sql
SELECT d.DepartmentName, COUNT(e.EmployeeID) AS total_employees
FROM employees AS e
JOIN departments AS d ON e.DepartmentID = d.DepartmentID
GROUP BY d.DepartmentName;

```


**Input:**
"Show the average salary for employees in each department."

**Output:**
```sql
SELECT d.DepartmentName, AVG(e.Salary) AS average_salary
FROM employees AS e
JOIN departments AS d ON e.DepartmentID = d.DepartmentID
GROUP BY d.DepartmentName;

```


**Input:**
"List all employees hired after January 1, 2023."

**Output:**
```sql
SELECT FirstName, LastName, HireDate
FROM employees
WHERE HireDate > '01-01-2023';

```


**Input:**
"Get the highest-paid employee in each department."

**Output:**
```sql
SELECT d.DepartmentName, e.FirstName, e.LastName, MAX(e.Salary) AS highest_salary
FROM employees AS e
JOIN departments AS d ON e.DepartmentID = d.DepartmentID
GROUP BY d.DepartmentName;

```


###OUTPUT FORMAT###
Return only the sqllite SQL query as a code block using triple backticks and the `sql` language tag, like this:

```sql
-- Your SQL query here
```
"""

In [ ]:
import json
def get_sql_query_via_gemini(genai_client, prompt, user_query):

  # https://www.geeksforgeeks.org/python/formatted-string-literals-f-strings-python/
  contents = f"""
  {prompt}

  Here's the user query in english you need to work on:
  {user_query}
  """
  response = genai_client.models.generate_content(model='gemini-2.5-flash', contents=contents)
  # print(response) # uncomment this and understand at the output

  # Access the usage_metadata attribute
  usage_metadata = response.usage_metadata

  # Print the different token counts
  print(f"Input Token Count: {usage_metadata.prompt_token_count}")
  print(f"Thoughts Token Count: {response.usage_metadata.thoughts_token_count}")
  print(f"Output Token Count: {usage_metadata.candidates_token_count}")
  print(f"Total Token Count: {usage_metadata.total_token_count}")

  output = response.text.replace('```sql', '').replace('```', '')

  return output


In [ ]:
import sqlite3
import pandas as pd

def execute_query(query, db_name='employee.db'):

    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Execute the query
        print(f"\nExecuting query on '{db_name}':\n{query}")
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        if not results:
          print("Query ran successfully but returned NO rows.")
          return pd.DataFrame()

        if cursor.description is None:
          print("No column metadata found.")
          return pd.DataFrame()

        columns = [desc[0] for desc in cursor.description]
        results_df = pd.DataFrame(results, columns=columns)

        return results_df


    except sqlite3.Error as e:
        print(f"Database error executing query: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None
    finally:
        if conn:
            conn.close()

In [ ]:
def text2sql(genai_client, prompt, user_query):
  output = get_sql_query_via_gemini(genai_client, prompt, user_query)
  results = execute_query(output)
  return results

In [ ]:
text2sql(genai_client, prompt,"Find employees who were hired in 2023.")

Input Token Count: 1069
Thoughts Token Count: 108
Output Token Count: 32
Total Token Count: 1209

Executing query on 'employee.db':

SELECT FirstName, LastName, HireDate
FROM employees
WHERE strftime('%Y', HireDate) = '2023';



,FirstName,LastName,HireDate
0,Ira,Haskett,2023-08-11
1,Bertram,Andriss,2023-07-18
2,Zora,Bampkin,2023-11-12
3,Sallyanne,Greenleaf,2023-04-12
4,Melisa,Will,2023-02-22
...,...,...,...
75,Pearline,Popple,2023-01-02
76,Mahmud,Wudeland,2023-01-28
77,Rodney,McAllen,2023-06-03
78,Lorry,Kaley,2023-03-11


In [ ]:
text2sql(genai_client, prompt,"Find employees who were hired in 2020.")

Input Token Count: 1069
Thoughts Token Count: 71
Output Token Count: 32
Total Token Count: 1172

Executing query on 'employee.db':

SELECT FirstName, LastName, HireDate
FROM employees
WHERE strftime('%Y', HireDate) = '2020';



,FirstName,LastName,HireDate
0,Lawton,McCrudden,2020-01-16
1,Gualterio,Streeton,2020-05-23
2,Waylan,Gatheridge,2020-01-16
3,Nanice,Calken,2020-02-01
4,Terra,Peizer,2020-07-16
...,...,...,...
83,Dorita,Middlehurst,2020-03-11
84,Rayner,Rutty,2020-09-26
85,Roxy,Honeywood,2020-12-02
86,Niles,Danhel,2020-09-24


In [ ]:
text2sql(genai_client, prompt,"List the top 5 highest-paid employees and their departments.")

Input Token Count: 1070
Thoughts Token Count: 68
Output Token Count: 54
Total Token Count: 1192

Executing query on 'employee.db':

SELECT e.FirstName, e.LastName, e.Salary, d.DepartmentName
FROM employees AS e
JOIN departments AS d ON e.DepartmentID = d.DepartmentID
ORDER BY e.Salary DESC
LIMIT 5;



,FirstName,LastName,Salary,DepartmentName
0,Tobi,Drees,149911.83,Services
1,Jeniece,Borzone,149736.63,Product Management
2,Winna,Simoneschi,149654.84,Sales
3,Noami,Flarity,149636.69,Training
4,Wally,Bergin,149460.62,Support


In [ ]:
text2sql(genai_client, prompt,"List employees and department name who work in 'Engineering' department.")

Input Token Count: 1070
Thoughts Token Count: 98
Output Token Count: 48
Total Token Count: 1216

Executing query on 'employee.db':

SELECT e.FirstName, e.LastName, d.DepartmentName
FROM employees AS e
JOIN departments AS d ON e.DepartmentID = d.DepartmentID
WHERE d.DepartmentName = 'Engineering';

Query ran successfully but returned NO rows.


""


In [ ]:
text2sql(genai_client, prompt,"Find the employees name with department and salary having 5th highest salary")

Input Token Count: 1071
Thoughts Token Count: 1015
Output Token Count: 74
Total Token Count: 2160

Executing query on 'employee.db':

SELECT e.FirstName, e.LastName, d.DepartmentName, e.Salary
FROM employees AS e
JOIN departments AS d ON e.DepartmentID = d.DepartmentID
WHERE e.Salary = (
    SELECT DISTINCT Salary
    FROM employees
    ORDER BY Salary DESC
    LIMIT 1 OFFSET 4
);



,FirstName,LastName,DepartmentName,Salary
0,Wally,Bergin,Support,149460.62


In [ ]:
text2sql(genai_client, prompt,"Find the employees name with department and salary with salary= 149460.62")

Input Token Count: 1078
Thoughts Token Count: 81
Output Token Count: 59
Total Token Count: 1218

Executing query on 'employee.db':

SELECT e.FirstName, e.LastName, d.DepartmentName, e.Salary
FROM employees AS e
JOIN departments AS d ON e.DepartmentID = d.DepartmentID
WHERE e.Salary = 149460.62;



,FirstName,LastName,DepartmentName,Salary
0,Wally,Bergin,Support,149460.62


In [ ]:
text2sql(genai_client, prompt,"Retrieve employees who are in departments with no employee hired in the last 2 years.")

Input Token Count: 1074
Thoughts Token Count: 1331
Output Token Count: 118
Total Token Count: 2523

Executing query on 'employee.db':

SELECT
  e.FirstName,
  e.LastName,
  e.HireDate,
  d.DepartmentName
FROM employees AS e
JOIN departments AS d
  ON e.DepartmentID = d.DepartmentID
WHERE
  e.DepartmentID IN (
    SELECT
      DepartmentID
    FROM departments
    WHERE
      DepartmentID NOT IN (
        SELECT DISTINCT
          DepartmentID
        FROM employees
        WHERE
          HireDate >= DATE('now', '-2 years')
      )
  );

Query ran successfully but returned NO rows.


""


In [ ]:
text2sql(genai_client, prompt,"Get the count of employees hired each year, sorted by year descending.")

Input Token Count: 1071
Thoughts Token Count: 81
Output Token Count: 45
Total Token Count: 1197

Executing query on 'employee.db':

SELECT strftime('%Y', HireDate) AS hire_year, COUNT(EmployeeID) AS employee_count
FROM employees
GROUP BY hire_year
ORDER BY hire_year DESC;



,hire_year,employee_count
0,2026,1
1,2025,106
2,2024,66
3,2023,80
4,2022,103
5,2021,97
6,2020,88
7,2019,94
8,2018,96
9,2017,83
